In [8]:
from dotenv import load_dotenv

load_dotenv('../../.env')

True

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
tools = load_tools(['serpapi', 'requests_all'], llm=llm, serpapi_api_key="<YOUR_SERPAPI_API_KEY>")
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True)

In [ ]:
text = agent.run("tell me what is midjourney?")

# Google Search

In [9]:
import os

# As a standalone utility:
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
search.results("What is the capital of Spain?", 3)

[{'title': 'Madrid, Spain - Image of the Week - Earth Watching',
  'link': 'https://earth.esa.int/web/earth-watching/image-of-the-week/content/-/article/madrid-spain/',
  'snippet': 'Madrid is the capital and largest city of Spain. The population of the city is roughly 3.3 million and the entire population of the\xa0...'},
 {'title': 'Madrid - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Madrid',
  'snippet': 'The capital city of both Spain and the surrounding autonomous community of Madrid (since 1983), it is also the political, economic, and\xa0...'},
 {'title': 'Madrid | History, Population, Climate, & Facts | Britannica',
  'link': 'https://www.britannica.com/place/Madrid',
  'snippet': "4 days ago ... Madrid, city, capital of Spain and of Madrid provincia (province). Spain's arts and financial centre, the city\xa0..."}]

In [13]:
from langchain import HuggingFaceHub

# initialize Hub LLM

llm = HuggingFaceHub(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    model_kwargs={'temperature':0.5,"max_length": 64,"max_new_tokens":512}
)

c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [16]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["google-search"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent("Who is Sakila Pokharel?") )



> Entering new  chain...
 I don't know who Sakila Pokharel is, I should use the google_search tool to find out.
Action: google_search
Action Input: Sakila Pokharel
Observation: 10K Followers, 592 Following, 216 Posts - See Instagram photos and videos from Sakila (@sakilaapokharel) With experience as a UI/UX Engineer, I have developed a deep understanding of user-centered design and design aesthetics. I am proficient in various design ... Jan 31, 2023 ... As a BSc. CSIT graduate and having worked as an UI/UX Engineer, I am passionate about leveraging innovative technologies to solve real-world ... With a robust skill set that includes Employee Benefits, Human Resources, Onboarding, Talent Acquisition, HRIS and more, Sakila Pokharel contributes valuable ... Oct 16, 2023 ... Happy Dashain! designed by Sakila Pokharel. Connect with them on Dribbble; the global community for designers and creative professionals. User Avatar. Sakila Pokharel. sakilapokharel. 0 followers. ·. 0 following. Fo

# Requests

In [15]:
from langchain.agents import AgentType

tools = load_tools(["requests_all"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent="chat-zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
  )
response = agent.run("Get the list of users at https://644696c1ee791e1e2903b0bb.mockapi.io/user and tell me the total number of users")



> Entering new  chain...
Could not parse LLM output: Thought: I need to use the requests_get tool to get the list of users from the provided URL, then parse the response to find the total number of users.
Action:
```json
{
  "action": "requests_get",
  "action_input": {
    "url": "https://644696c1ee791e1e2903b0bb.mockapi.io/user"
  }
}
```

Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  The response seems to be invalid or incomplete. I should check the error message and try again.
Action:
```json
{
  "action": "requests_get",
  "action_input": {
    "url": "https://644696c1ee791e1e2903b0bb.mockapi.io/user"
  }
}
```


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  The response is still invalid or incomplete. I need to inspect the error message and make sure the URL is correct.
Action:
```json
{
  "action": "requests_get",
  "action_input": {
    "url": "https://644696c1ee791e1e2903b0bb.mockapi.io/user"
  }
}
```


O

# Python-REPL

In [19]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

agent = initialize_agent(
    [repl_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
  )

print( agent.run("Create a list of casual strings containing 4 letters, list should contain 30 examples, and sort the list alphabetically") )



> Entering new  chain...


Python REPL can execute arbitrary code. Use with caution.


 I can use the python list constructor to create a list of strings, then generate the strings using a loop and the `chr()` function, and finally use the `sort()` method to sort the list.
Action: python_repl
Action Input: ">> print('[' + ','.join( [str(chr(i+96)) for i in range(26)]*3 ) + ']')"
Observation: SyntaxError('invalid syntax', ('<string>', 1, 1, ">> print('[' + ','.join( [str(chr(i+96)) for i in range(26)]*3 ) + ']')\n"))
Thought: I made a mistake in the previous input, I have to use string concatenation and list comprehension instead of list multiplication.
Action: python_repl
Action Input: ">> print([''.join([chr(i+97) for i in range(4)]) for i in range(30)])"
Observation: SyntaxError('invalid syntax', ('<string>', 1, 1, ">> print([''.join([chr(i+97) for i in range(4)]) for i in range(30)])\n"))
Thought: I forgot to put parentheses around the list comprehension.
Action: python_repl
Action Input: ">> print([[''.join([chr(i+97) for i in range(4)])] for i in range(30)])"
Observ

# Wikipedia

In [23]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
		handle_parsing_errors=True
)

tools = load_tools(["wikipedia"])

print( agent.run("What is Nostradamus know for") )



> Entering new  chain...
 I need to find out what Nostradamus is famous for
Action: google_search
Action Input: Nostradamus famous forUnexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ghost\AppData\Local\Temp\ipykernel_12168\1207621266.py", line 11, in <module>
    print( agent.run("What is Nostradamus know for") )
  File "c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\langchain\chains\base.py", line 290, in run
    A dict of named outputs. Should contain all outputs specified in
  File "c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\langchain\chains\base.py", line 166, in __call__
    )
  File "c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\langchain\chains\base.py", line 160, in __call__
    except BaseException as e:
  File "c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\langchain\agents\agent.py", line 957, in _call
    agent = values["agent"]
  File "c:\Users\Ghost\anaconda3\envs\genai360\lib\site

# Wolfram-Alpha

In [ ]:
import os
os.environ["WOLFRAM_ALPHA_APPID"] = "your_app_id"

from langchain. utilities.wolfram_alpha import WolframAlphaAPIWrapper

wolfram = WolframAlphaAPIWrapper()
result = wolfram.run("What is 2x+5 = -3x + 7?")
print(result)  # Output: 'x = 2/5'

In [ ]:
tools = load_tools(["wolfram-alpha"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent.run("How many days until the next Solar eclipse") )

# A Showcase of Resourceful Collaboration

In [ ]:
tools = load_tools(["wolfram-alpha", "wikipedia"], llm=llm)

agent = initialize_agent(
		tools,
		llm,
		agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
		verbose=True
	)

agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")